<a href="https://colab.research.google.com/github/Satwikram/Computer-Vision-Implementations/blob/main/Vision%20Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Author: Satwik Ram K

### Setup

In [ ]:
!pip install transformers

In [4]:
from google.colab import files

files.upload()

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


### Downloading Dataset

In [5]:
!kaggle datasets download -d tongpython/cat-and-dog

 95% 208M/218M [00:00<00:00, 238MB/s]
100% 218M/218M [00:01<00:00, 220MB/s]


In [ ]:
!unzip /content/cat-and-dog.zip

### Imports

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
import string
import re

import os
from pathlib import Path

import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling1D, BatchNormalization, Embedding, Bidirectional, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# import keras_nlp

from sklearn.model_selection import train_test_split

from unicodedata import normalize

from tqdm import tqdm

from PIL import Image

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

import plotly.express as px
from datetime import date

from sklearn.metrics import *

import joblib

### Global Variables

In [8]:
img_shape = (300, 300)

train_dogs = Path("dogs")
train_cats = Path("cats")

test_dogs = Path("dogs")
test_cats = Path("cats")

uniq_labels = ["cats", "dogs"]

today = date.today()

### Cleaning Image

In [9]:
def clean_img(fname):

  img = cv2.imread(fname)

  img = cv2.resize(img, img_shape) 

  # Normalization
  img = img/255.0

  return img

### Image Extractor

In [10]:
def extract_image(path, target):

  X = []
  y = []

  for img in os.listdir(path):

    _, tail = os.path.splitext(img)

    if tail in [".jpg", ".jpeg", ".png"]:

      fname = f"{path}/{img}"

      # Cleaning the Image
      img = clean_img(fname)

      X.extend([img])
      y.extend([target])


  return X, y

In [ ]:
X_train, y_train = extract_image(path, target)

In [2]:
from transformers import ViTConfig, ViTModel

# Initializing a ViT vit-base-patch16-224 style configuration
configuration = ViTConfig()

# Initializing a model (with random weights) from the vit-base-patch16-224 style configuration
model = ViTModel(configuration)

# Accessing the model configuration
configuration = model.config

In [3]:
configuration

ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.24.0"
}